In [1]:
from __future__ import print_function
import pandas as pd

pd.set_option('precision',5)
pd.options.display.max_rows = 10

In [2]:
import numpy as np

In [3]:
from recordlinkage.datasets import binary_vectors

In [4]:
#start with importing the recordlinkage module
import recordlinkage as rl

In [5]:
data = pd.read_csv("/Users/derri/Onedrive/ml/Cleaned1.csv")

In [6]:
# make record pairs #
# It is very intuitive to start with comparing each record in the dataset with all other records# 
#Indexing: Each record pair should contain two different records of the dataset.
indexer = rl.Index()
indexer.full()
candidate_links = indexer.index(data)

In [7]:
print (len(data), len(candidate_links))
# (20032*20032-20032)/2 = 200630496

20032 200630496


In [8]:
data.head()

,EnterpriseID,newLAST,newFIRST,DOB,GENDER,SSN,newZIP,concatADDRESS
0,14744718,FERGUSON,SANDRA,NaN,F,887043281,10462.0,1METROPOLITAN
1,15444141,BEEBE,CLYDE,1959/12/24,M,845830508,10002.0,100PITT
2,15740881,BOUSTANY,MARIA,1921/7/6,F,834882296,10452.0,1008UNIVERSTY
3,15586154,NGUYEN,NATHANIEL,1935/12/16,M,878125866,11205.0,102CUMBERLAND
4,15563691,MOEN,LORI,1938/4/13,F,868511569,10453.0,1020GRANDCONCOURSE


In [9]:
#to reduce the number of pairs#
indexer_new = rl.Index()
indexer_new.block('newFIRST') #newFIRST is the blocking variable
candidate_links_new = indexer_new.index(data)

In [10]:
print (len(candidate_links_new))

335326


In [11]:
#how to compare attributes#
compare_cl = rl.Compare()

compare_cl.string('newFIRST', 'newFIRST', method='jarowinkler')
compare_cl.string('newLAST', 'newLAST', method='jarowinkler')
compare_cl.exact('DOB', 'DOB', label='DOB')
compare_cl.string('SSN', 'SSN', label='SSN')
compare_cl.exact('GENDER', 'GENDER', label='GENDER')
compare_cl.numeric('newZIP', 'newZIP', label='newZIP')
compare_cl.string('concatADDRESS', 'concatADDRESS', method='levenshtein')

features = compare_cl.compute(candidate_links_new, data)

In [12]:
features.head() #The first 10 comparison vectors#

0        1  DOB      SSN  GENDER  newZIP        6
201 0    1.0  0.41667    0  0.11111       1     0.0  0.15385
204 0    1.0  0.56944    0  0.11111       1     0.0  0.15385
    201  1.0  0.40741    1  1.00000       1     1.0  0.91667
487 0    1.0  0.44167    0  0.22222       0     0.0  0.00000
    201  1.0  0.48333    0  0.11111       0     0.0  0.08333

In [13]:
features.describe().T

,count,mean,std,min,25%,50%,75%,max
0,335326.0,1.00000,0.00000,1.0,1.00000,1.00000,1.00000,1.0
1,335326.0,0.40389,0.21900,0.0,0.40067,0.45556,0.53571,1.0
DOB,335326.0,0.02018,0.14061,0.0,0.00000,0.00000,0.00000,1.0
SSN,335326.0,0.24399,0.14632,0.0,0.11111,0.22222,0.33333,1.0
GENDER,335326.0,0.68345,0.46513,0.0,0.00000,1.00000,1.00000,1.0
newZIP,335326.0,0.02865,0.15538,0.0,0.00000,0.00000,0.00000,1.0
6,335326.0,0.21544,0.18776,0.0,0.06667,0.16667,0.37500,1.0


In [14]:
# Sum the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.98000       2
6.97500       1
6.97143       1
6.96667       2
6.96000       5
           ... 
1.16374      28
1.16111       7
1.15873       3
1.15657       2
1.11111    1078
Length: 39293, dtype: int64

In [15]:
#Classification Step
matches = features[features.sum(axis=1) > 3]

print(len(matches))
matches.head(10)

74793


0        1  DOB      SSN  GENDER  newZIP        6
204  201   1.0  0.40741    1  1.00000       1     1.0  0.91667
614  201   1.0  0.59524    0  0.22222       1     0.0  0.18750
1012 201   1.0  0.48333    0  0.33333       1     0.5  0.14286
     204   1.0  0.43704    0  0.33333       1     0.5  0.09524
1143 1012  1.0  0.63333    0  0.22222       1     0.0  0.31818
1446 1012  1.0  0.57778    0  0.22222       1     0.0  0.38095
     1143  1.0  0.47222    0  0.22222       1     0.0  0.40909
1932 487   1.0  0.67619    0  0.11111       0     1.0  0.23077
     1012  1.0  0.56190    0  0.11111       1     0.0  0.33333
     1143  1.0  0.59524    0  0.33333       1     0.0  0.27273

In [59]:
matchesh = features[features.sum(axis=1) > 4.5]

print(len(matchesh))
matchesh.head(10)

5238


,,0,1,DOB,SSN,GENDER,newZIP,6
204,201,1.0,0.40741,1,1.0,1,1.0,0.91667
4692,4691,1.0,0.47778,1,1.0,1,1.0,1.00000
4803,4802,1.0,1.00000,1,1.0,0,1.0,1.00000
4824,3388,1.0,0.47222,1,1.0,1,0.0,1.00000
5357,1012,1.0,0.76000,1,1.0,1,1.0,0.76190
7022,7021,1.0,0.00000,1,1.0,1,1.0,0.61111
10090,9962,1.0,0.00000,1,1.0,1,0.0,0.85714
10346,1143,1.0,0.59524,1,1.0,1,0.0,0.17391
12098,12097,1.0,0.90833,1,1.0,1,1.0,1.00000
12267,487,1.0,1.00000,1,1.0,0,1.0,0.50000


In [16]:
kmeans = rl.KMeansClassifier()

In [17]:
result_kmeans = kmeans.fit_predict(features)

In [18]:
len(result_kmeans)

6756

In [19]:
result_kmeans

MultiIndex(levels=[[33, 42, 61, 66, 76, 82, 93, 96, 108, 109, 111, 120, 122, 129, 130, 131, 132, 133, 137, 139, 141, 144, 145, 146, 154, 155, 160, 174, 175, 176, 180, 197, 199, 201, 203, 204, 205, 207, 219, 233, 236, 238, 239, 244, 245, 247, 254, 259, 262, 267, 272, 273, 286, 294, 296, 298, 302, 306, 308, 310, 314, 316, 320, 321, 323, 324, 330, 333, 334, 340, 341, 344, 347, 348, 349, 352, 353, 355, 356, 358, 360, 361, 363, 364, 365, 366, 369, 371, 373, 376, 377, 379, 383, 385, 393, 395, 397, 401, 407, 411, 412, 415, 416, 417, 421, 422, 423, 426, 433, 434, 437, 443, 445, 446, 447, 448, 452, 453, 457, 462, 464, 466, 467, 468, 471, 472, 474, 480, 481, 482, 483, 484, 487, 488, 492, 495, 497, 499, 502, 510, 511, 513, 514, 517, 521, 522, 524, 525, 526, 527, 530, 531, 533, 534, 535, 537, 538, 539, 540, 543, 544, 546, 550, 552, 559, 565, 567, 568, 569, 570, 586, 591, 594, 596, 598, 600, 603, 604, 606, 610, 611, 613, 614, 618, 619, 620, 621, 622, 630, 631, 632, 635, 643, 646, 647, 648, 651, 653

In [60]:
rl.confusion_matrix(matchesh, result_kmeans, len(features))

array([[  5133,    105],
       [  1623, 328465]])

In [62]:
rl.fscore(matchesh, result_kmeans)

0.855927963981991

In [64]:
rl.accuracy(matchesh,result_kmeans,len(features))

0.9948468057949578

In [68]:
print(rl.precision(matchesh,result_kmeans))
print(rl.recall(matchesh,result_kmeans))

0.7597690941385435
0.9799541809851088


In [28]:
from recordlinkage.base import BaseCompareFeature

In [47]:
class CompareZipCodes(BaseCompareFeature):

    def _compute_vectorized(self, s1, s2):
        """Compare zipcodes.

        If the zipcodes in both records are identical, the similarity
        is 0. If the first two values agree and the last three don't, then
        the similarity is 0.5. Otherwise, the similarity is 0.
        """

        # check if the zipcode are identical (return 1 or 0)
        sim = (s1 == s2).astype(float)

        # check the first 2 numbers of the distinct comparisons
        sim[(sim == 0) & (s1.astype(str).str[0:2] == s2.astype(str).str[0:2])] = 0.5

        return sim

In [48]:
comparer = rl.Compare()
comparer.string('newFIRST', 'newFIRST', method='jarowinkler')
comparer.string('newLAST', 'newLAST', method='jarowinkler')
comparer.exact('DOB', 'DOB', label='DOB')
comparer.string('SSN', 'SSN', label='SSN')
comparer.exact('GENDER', 'GENDER', label='GENDER')
comparer.string('concatADDRESS', 'concatADDRESS', method='levenshtein')
comparer.add(CompareZipCodes('newZIP', 'newZIP', label='newZIP'))

In [49]:
features1 = comparer.compute(candidate_links_new, data)

In [50]:
features1.head()

0        1  DOB      SSN  GENDER        5  newZIP
201 0    1.0  0.41667    0  0.11111       1  0.15385     0.5
204 0    1.0  0.56944    0  0.11111       1  0.15385     0.5
    201  1.0  0.40741    1  1.00000       1  0.91667     1.0
487 0    1.0  0.44167    0  0.22222       0  0.00000     0.5
    201  1.0  0.48333    0  0.11111       0  0.08333     0.5

In [51]:
features1.describe().T

,count,mean,std,min,25%,50%,75%,max
0,335326.0,1.00000,0.00000,1.0,1.00000,1.00000,1.00000,1.0
1,335326.0,0.40389,0.21900,0.0,0.40067,0.45556,0.53571,1.0
DOB,335326.0,0.02018,0.14061,0.0,0.00000,0.00000,0.00000,1.0
SSN,335326.0,0.24399,0.14632,0.0,0.11111,0.22222,0.33333,1.0
GENDER,335326.0,0.68345,0.46513,0.0,0.00000,1.00000,1.00000,1.0
5,335326.0,0.21544,0.18776,0.0,0.06667,0.16667,0.37500,1.0
newZIP,335326.0,0.22874,0.26961,0.0,0.00000,0.00000,0.50000,1.0


In [52]:
# Sum the comparison results.
features1.sum(axis=1).value_counts().sort_index(ascending=False)

6.98000      2
6.97500      1
6.97143      1
6.96667      2
6.96000      5
          ... 
1.16667     50
1.16374     15
1.16111      5
1.15873      2
1.11111    954
Length: 46907, dtype: int64

In [53]:
#Classification Step
matches1 = features1[features1.sum(axis=1) > 3]

print(len(matches1))
matches1.head(10)

125441


0        1  DOB      SSN  GENDER        5  newZIP
201  0    1.0  0.41667    0  0.11111       1  0.15385     0.5
204  0    1.0  0.56944    0  0.11111       1  0.15385     0.5
     201  1.0  0.40741    1  1.00000       1  0.91667     1.0
614  0    1.0  0.60714    0  0.11111       1  0.12500     0.5
     201  1.0  0.59524    0  0.22222       1  0.18750     0.5
     204  1.0  0.47619    0  0.22222       1  0.25000     0.5
1012 0    1.0  0.38333    0  0.22222       1  0.14286     0.5
     201  1.0  0.48333    0  0.33333       1  0.14286     0.5
     204  1.0  0.43704    0  0.33333       1  0.09524     0.5
     614  1.0  0.44762    0  0.22222       1  0.14286     0.5

In [75]:
matchesh1 = features[features.sum(axis=1) > 4.5]

print(len(matchesh1))
matchesh1.head(10)

5238


,,0,1,DOB,SSN,GENDER,newZIP,6
204,201,1.0,0.40741,1,1.0,1,1.0,0.91667
4692,4691,1.0,0.47778,1,1.0,1,1.0,1.00000
4803,4802,1.0,1.00000,1,1.0,0,1.0,1.00000
4824,3388,1.0,0.47222,1,1.0,1,0.0,1.00000
5357,1012,1.0,0.76000,1,1.0,1,1.0,0.76190
7022,7021,1.0,0.00000,1,1.0,1,1.0,0.61111
10090,9962,1.0,0.00000,1,1.0,1,0.0,0.85714
10346,1143,1.0,0.59524,1,1.0,1,0.0,0.17391
12098,12097,1.0,0.90833,1,1.0,1,1.0,1.00000
12267,487,1.0,1.00000,1,1.0,0,1.0,0.50000


In [54]:
result1_kmeans = kmeans.fit_predict(features1)

In [55]:
len(result1_kmeans)

102565

In [56]:
result1_kmeans

MultiIndex(levels=[[33, 42, 61, 66, 76, 82, 93, 96, 108, 109, 111, 120, 122, 129, 130, 131, 132, 133, 137, 139, 141, 144, 145, 146, 154, 155, 160, 174, 175, 176, 180, 197, 199, 201, 203, 204, 205, 207, 219, 233, 236, 238, 239, 244, 245, 247, 254, 259, 262, 267, 272, 273, 286, 294, 296, 298, 302, 306, 308, 310, 314, 316, 320, 321, 323, 324, 330, 333, 334, 340, 341, 344, 347, 348, 349, 352, 353, 355, 356, 358, 360, 361, 363, 364, 365, 366, 369, 371, 373, 376, 377, 379, 383, 385, 393, 395, 397, 401, 407, 411, 412, 415, 416, 417, 421, 422, 423, 426, 433, 434, 437, 443, 445, 446, 447, 448, 452, 453, 457, 462, 464, 466, 467, 468, 471, 472, 474, 480, 481, 482, 483, 484, 487, 488, 492, 495, 497, 499, 502, 510, 511, 513, 514, 517, 521, 522, 524, 525, 526, 527, 530, 531, 533, 534, 535, 537, 538, 539, 540, 543, 544, 546, 550, 552, 559, 565, 567, 568, 569, 570, 586, 591, 594, 596, 598, 600, 603, 604, 606, 610, 611, 613, 614, 618, 619, 620, 621, 622, 630, 631, 632, 635, 643, 646, 647, 648, 651, 653

In [76]:
rl.confusion_matrix(matchesh1, result1_kmeans, len(features1))

array([[  4891,    347],
       [ 97674, 232414]])

In [77]:
print(rl.precision(matchesh1,result1_kmeans))
print(rl.recall(matchesh1,result1_kmeans))
print(rl.fscore(matchesh1, result1_kmeans))

0.04768683274021352
0.9337533409698359
0.09073958980733375


In [78]:
rl.accuracy(matchesh1, result1_kmeans, len(features1))

0.7076844622844635